Этот бот записывает рецепты и создает кулинарную книгу. У него есть кнопки для добавления, удаления, редактирования, вывода рецепта, вывода списков рецепта, выставления оценки рецепта. Также боту можно направить запрос: Вдохнови меня! И бот выберет одну из своих цитат про еду из фильмов/мультфильмов.

In [17]:
!pip install telebot

In [2]:
import telebot
from telebot import types
import random
import time
import sqlite3

In [3]:
# токен бота
token = '8326246362:AAG_ufSxHxL1CN2bxtpqREeBbE2CI6t9guQ'

In [4]:
# активация бота
bot = telebot.TeleBot(token, parse_mode=None, threaded=False)
# цитаты (нужны для будущей функции)
quotes = ['Мы должны есть, чтобы жить, а не жить, чтобы есть 🍔',
          'Еда — это истинное народное искусство', 'Хороший аппетит — лучший соус 🍰',
          'Таити, Таити — нас и здесь неплохо кормят', 'Минутoчку! За чей счет этoт банкет? Ктo oплачивать будет? 🍕',
          'Почки заячьи верченые, головы щучьи с чесноком, икра черная, красная. Заморская икра — баклажанная! Красота! 🍲',
          'Соль — это всё! Половина успеха — правильно посолить. Ой, это же сахар! Ну, не важно 🥗']

In [5]:
# подключаем базу данных
conn = sqlite3.connect('cooking.db', check_same_thread=False)
# курсор для работы с таблицами
cursor = conn.cursor()

In [6]:
try:
    # sql запрос для создания таблицы
    query = ('CREATE TABLE "recipes" '
         '("ID" INTEGER UNIQUE, "user_id" INTEGER, "recipe_name" TEXT, "recipe" TEXT, "rating" INT DEFAULT 0,'
         'PRIMARY KEY ("ID"))')
    cursor.execute(query)
    conn.commit()
except:
    pass

In [7]:
# эта добавляет строчку с рецептом
def add_recipe(msg):
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO recipes (user_id, recipe_name, recipe) VALUES (?, ?, ?)',
                          (msg.from_user.id, msg.text.split('Название: ')[1], msg.text.split('Название: ')[0]))
        con.commit()
    bot.send_message(msg.chat.id, 'Рецепт записан 🙌')
    send_keyboard(msg, "Чем еще могу помочь? ✨")

In [8]:
# добавляем рейтинг
def add_rating(msg):
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('UPDATE recipes SET rating = ? WHERE recipe_name == ? AND user_id = ?',
                      (msg.text.split()[0], ' '.join(msg.text.split()[1:]), msg.from_user.id))
        con.commit()
    bot.send_message(msg.chat.id, 'Рейтинг выставлен 🙌')
    send_keyboard(msg, "Чем еще могу помочь? ✨")

In [9]:
# корректировка рецепта
def correct_recipe(msg):
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('UPDATE recipes SET recipe = ? WHERE recipe_name = ? AND user_id = ?',
                      (msg.text.split('Название: ')[0], msg.text.split('Название: ')[1], msg.from_user.id))
        con.commit()
    bot.send_message(msg.chat.id, 'Рецепт исправлен 🙌')
    send_keyboard(msg, "Чем еще могу помочь? ✨")

In [10]:
# функция для вывода списка рецептов с рейтингом
def get_recipe_string(recipes):
    recipe_str = []
    for val in list(enumerate(recipes)):
        recipe_str.append(str(val[0] + 1) + '. ' + val[1][0] + '\n' + 'рейтинг: ' + str(val[1][1]) + '\n')
    return ''.join(recipe_str)

# отправляем список рецептов
def show_recipes(msg):
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT recipe_name, rating FROM recipes WHERE user_id=={}'.format(msg.from_user.id))
        s = cursor.fetchall()
        if not s:
            bot.send_message(msg.chat.id, 'У вас пока нет рецептов 🙌')
        else:
            recipes = get_recipe_string(s)
            bot.send_message(msg.chat.id, recipes)
        send_keyboard(msg, "Чем еще могу помочь? ✨")

In [11]:
# выделяет рецепт для удаления
def delete_one_recipe(msg):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT recipe_name FROM recipes WHERE user_id=={}'.format(msg.from_user.id))
        recipes = cursor.fetchall()
        if not recipes:
            bot.send_message(msg.chat.id, 'Нет рецептов для удаления 🙌')
            send_keyboard(msg, "Чем еще могу помочь? ✨")
            return
        for value in recipes:
            markup.add(types.KeyboardButton(value[0]))
        msg = bot.send_message(msg.from_user.id,
                               text = "Выбери один рецепт ✨",
                               reply_markup=markup)
        bot.register_next_step_handler(msg, delete_one_recipe_)

# удаляет рецепт
def delete_one_recipe_(msg):
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('DELETE FROM recipes WHERE user_id==? AND recipe_name==?', (msg.from_user.id, msg.text))
        con.commit()
    bot.send_message(msg.chat.id, 'Вы удалили рецепт 🙌')
    send_keyboard(msg, "Чем еще могу помочь? ✨")

In [12]:
# показываем рецепт
def show_recipe(msg):
    with sqlite3.connect('cooking.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT recipe FROM recipes WHERE user_id==? AND recipe_name =?', (msg.from_user.id, msg.text))
        s = cursor.fetchall()
        if s:
            bot.send_message(msg.chat.id, s[0][0])
        else:
            bot.send_message(msg.chat.id, 'Такого рецепта нет 🙌')
        send_keyboard(msg, "Чем еще могу помочь? ✨")

In [13]:
# привязываем функции к кнопкам
def callback_worker(call):
    if call.text == "Добавить рецепт":
        msg = bot.send_message(call.chat.id, 'Давайте добавим рецепт! Напишите его в чат в формате "Текст рецепта. Название: текст названия"')
        bot.register_next_step_handler(msg, add_recipe)

    elif call.text == "Список моих рецептов":
        try:
            show_recipes(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Придумайте рецепт и добавьте 🙌')
            send_keyboard(call, "Чем еще могу помочь? ✨")

    elif call.text == "Удалить рецепт":
        try:
            delete_one_recipe(call)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Придумайте рецепт и добавьте 🙌')
            send_keyboard(call, "Чем еще могу помочь? ✨")

    elif call.text == "Оценить рецепт":
        try:
            msg = bot.send_message(call.chat.id, 'Давайте оценим рецепт! Напишите рейтинг от 0 до 5 по формату "Оценка рейтинга текст названия"')
            bot.register_next_step_handler(msg, add_rating)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Придумайте рецепт и оцените его 🙌')
            send_keyboard(call, "Чем еще могу помочь? ✨")

    elif call.text == "Редактировать рецепт":
        msg = bot.send_message(call.chat.id, 'Давайте скорректируем рецепт! Напишите его в чат в формате "Текст рецепта. Название: текст названия"')
        bot.register_next_step_handler(msg, correct_recipe)

    elif call.text == "Выбор рецепта из моей книги":
        try:
            msg = bot.send_message(call.chat.id, 'Напишите название рецепта')
            bot.register_next_step_handler(msg, show_recipe)
        except:
            bot.send_message(call.chat.id, 'Здесь пусто. Придумайте рецепт и добавьте 🙌')
            send_keyboard(call, "Чем еще могу помочь? ✨")

    elif call.text == 'Вдохнови меня!':
        handle_message(call)

    elif call.text == "Спасибо! Я сделал(а), что хотел(а)":
        bot.send_message(call.chat.id, 'Хорошего дня!🥰 Когда захотите продолжнить нажмите на команду /start')

In [14]:
bot_running = True

In [15]:
# делаем кнопки
@bot.message_handler(commands=['start', 'help'])
def send_keyboard(message, text="Привет, я твоя кулинарная книга📚. Что я могу сделать для тебя?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('Добавить рецепт')
    itembtn2 = types.KeyboardButton('Список моих рецептов')
    itembtn3 = types.KeyboardButton('Удалить рецепт')
    itembtn4 = types.KeyboardButton("Оценить рецепт")
    itembtn5 = types.KeyboardButton('Редактировать рецепт')
    itembtn6 = types.KeyboardButton('Выбор рецепта из моей книги')
    itembtn7 = types.KeyboardButton('Спасибо! Я сделал(а), что хотел(а)')
    keyboard.add(itembtn1, itembtn2, itembtn3)
    keyboard.add(itembtn4, itembtn5, itembtn6)
    keyboard.add(itembtn7)

    msg = bot.send_message(message.from_user.id,
                     text=text, reply_markup=keyboard)
    bot.register_next_step_handler(msg, callback_worker)

# запрос на вдохновение
@bot.callback_query_handler(func=lambda call: call.text == 'Вдохнови меня!')
def handle_message(call):
    q = random.choice(quotes)
    markup = types.InlineKeyboardMarkup(row_width=2)
    bot.send_message(call.chat.id, q, reply_markup=markup)
    send_keyboard(call, "Нужно что-то еще? ✨")

In [16]:
# запуск бота
bot.polling(none_stop=True, interval=2, timeout=60)